In [12]:
%load_ext autoreload
%autoreload 2
import sys
import os
current_dir = os.getcwd()
libs_path = os.path.abspath(os.path.join(current_dir, "..", "libs"))
if libs_path not in sys.path:
    sys.path.append(libs_path)

import numpy as np
import pandas as pd
pd.set_option('display.precision', 3)
import data_processing as dp
import models as myML
from typing import List, Optional, Tuple

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Flatten, Dropout, RepeatVector, TimeDistributed, UpSampling1D
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import os
import warnings

warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

# Установка seed для воспроизводимости
np.random.seed(1337)
tf.random.set_seed(1337)





dataset = pd.read_excel("../datasets_ref/source.xls")

dataset["temp_Class"] = np.uint(np.bool_(dataset["Class"]))
dataset["Class"] = dataset["temp_Class"]
dataset = dataset.drop(columns=["temp_Class"])

dataset["power, W"] = dataset["Ubs,V"] * dataset["Ibs,A"]
dataset["D+"] = np.sqrt(np.power(dataset["TR11,C"],2) + np.power(dataset["TR13,C"], 2) + np.power(dataset["TR15,C"], 2))
dataset["D-"] = np.sqrt(np.power(dataset["TR12,C"],2) + np.power(dataset["TR14,C"], 2) + np.power(dataset["TR16,C"], 2))
dataset["D"] = np.sqrt(np.power(dataset["D+"],2) + np.power(dataset["D-"], 2))








X = dataset.drop(columns=['Class']).values
y = dataset['Class'].values

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, confusion_matrix
import random
import operator
import warnings
warnings.filterwarnings('ignore')

# DEAP imports
from deap import base, creator, tools, algorithms

# ============================================================================
# 1. FIXED ARCHITECTURE HYPERPARAMETERS (empirically set)
# ============================================================================
N_CONV_LAYERS = 2          # Fixed number of Conv1D layers
N_RECURRENT_LAYERS = 2     # Fixed number of Bidirectional LSTM layers
N_DENSE_LAYERS = 1         # Fixed number of dense layers in bottleneck/decoder
WINDOW_SIZE = 50           # Fixed time series window size
N_FEATURES = None          # Will be set after data loading

# ============================================================================
# 2. DATA PREPROCESSING
# ============================================================================
def create_windows(data, window_size=WINDOW_SIZE):
    """Create sliding windows from time series data."""
    X = []
    for i in range(len(data) - window_size + 1):
        X.append(data[i:i + window_size])
    return np.array(X)

def preprocess_data(df, target_col='class', test_ratio=0.2, val_ratio=0.1):
    """
    Preprocess dataset:
    - Splits into train (unlabeled), validation (unlabeled), test (labeled)
    - Normalizes using training statistics ONLY
    - Ignores 'class' column during autoencoder training
    """
    global N_FEATURES
    
    # Separate features (exclude target if exists)
    if target_col in df.columns:
        features = df.drop(columns=[target_col]).values
        labels = df[target_col].values if target_col in df.columns else None
    else:
        features = df.values
        labels = None
    
    N_FEATURES = features.shape[1]
    
    # Create windows
    X_windows = create_windows(features)
    
    # Split indices (preserve temporal order)
    n_total = len(X_windows)
    n_test = int(n_total * test_ratio)
    n_val = int(n_total * val_ratio)
    n_train = n_total - n_test - n_val
    
    X_train = X_windows[:n_train]
    X_val = X_windows[n_train:n_train + n_val]
    X_test = X_windows[n_train + n_val:]
    
    # Extract labels for test set windows (use last timestep label)
    y_test = None
    if labels is not None:
        y_test_windows = create_windows(labels.reshape(-1, 1))
        y_test = y_test_windows[n_train + n_val:, -1, 0].astype(int)
    
    # Normalize using training statistics only
    scaler = StandardScaler()
    X_train_flat = X_train.reshape(-1, N_FEATURES)
    scaler.fit(X_train_flat)
    
    X_train_norm = scaler.transform(X_train_flat).reshape(X_train.shape)
    X_val_norm = scaler.transform(X_val.reshape(-1, N_FEATURES)).reshape(X_val.shape)
    X_test_norm = scaler.transform(X_test.reshape(-1, N_FEATURES)).reshape(X_test.shape)
    
    return (X_train_norm, X_val_norm, X_test_norm, y_test, scaler)

# ============================================================================
# 3. AUTOENCODER MODEL BUILDER (parameterized)
# ============================================================================
def build_autoencoder(hparams):
    """
    Build convolutional-Bidirectional LSTM autoencoder.
    
    hparams dict keys:
        - conv_filters: list of ints (length = N_CONV_LAYERS)
        - kernel_sizes: list of ints (length = N_CONV_LAYERS)
        - lstm_units: list of ints (length = N_RECURRENT_LAYERS)
        - dense_units: int (bottleneck size)
        - optimizer: str ('adam', 'rmsprop', 'sgd')
    """
    # ===== ENCODER =====
    inputs = layers.Input(shape=(WINDOW_SIZE, N_FEATURES))
    x = inputs
    
    # Convolutional layers
    for i in range(N_CONV_LAYERS):
        x = layers.Conv1D(
            filters=hparams['conv_filters'][i],
            kernel_size=hparams['kernel_sizes'][i],
            activation='relu',
            padding='same'
        )(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
    
    # Recurrent layers
    for i in range(N_RECURRENT_LAYERS):
        return_sequences = (i < N_RECURRENT_LAYERS - 1)  # Only last layer returns single vector
        x = layers.Bidirectional(
            layers.LSTM(hparams['lstm_units'][i], return_sequences=return_sequences)
        )(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
    
    # Bottleneck
    encoded = layers.Dense(hparams['dense_units'], activation='relu', name='bottleneck')(x)
    
    # ===== DECODER =====
    x = layers.RepeatVector(WINDOW_SIZE)(encoded)
    
    # Recurrent layers (mirror encoder)
    for i in range(N_RECURRENT_LAYERS):
        return_sequences = True
        x = layers.Bidirectional(
            layers.LSTM(hparams['lstm_units'][i], return_sequences=return_sequences)
        )(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
    
    # Convolutional transpose layers (approximate inverse)
    for i in reversed(range(N_CONV_LAYERS)):
        x = layers.Conv1D(
            filters=hparams['conv_filters'][i],
            kernel_size=hparams['kernel_sizes'][i],
            activation='relu',
            padding='same'
        )(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
    
    # Output layer (reconstruct original features)
    outputs = layers.Conv1D(
        filters=N_FEATURES,
        kernel_size=3,
        activation='linear',
        padding='same'
    )(x)
    
    # Build model
    autoencoder = Model(inputs, outputs, name='ConvBiLSTM_Autoencoder')
    
    # Compile
    opt = keras.optimizers.get(hparams['optimizer'])
    autoencoder.compile(optimizer=opt, loss='mse')
    
    return autoencoder

# ============================================================================
# 4. GENETIC ALGORITHM WITH DEAP
# ============================================================================
def create_individual():
    """Create random individual representing hyperparameter set."""
    return [
        random.choice([16, 32, 64]),           # conv_filters layer 1
        random.choice([16, 32, 64]),           # conv_filters layer 2
        random.choice([3, 5, 7]),              # kernel_size layer 1
        random.choice([3, 5, 7]),              # kernel_size layer 2
        random.choice([32, 64, 128]),          # lstm_units layer 1
        random.choice([32, 64, 128]),          # lstm_units layer 2
        random.choice([16, 32, 64]),           # dense_units bottleneck
        random.choice(['adam', 'rmsprop'])     # optimizer
    ]

def individual_to_hparams(ind):
    """Convert DEAP individual to hyperparameter dictionary."""
    return {
        'conv_filters': [ind[0], ind[1]],
        'kernel_sizes': [ind[2], ind[3]],
        'lstm_units': [ind[4], ind[5]],
        'dense_units': ind[6],
        'optimizer': ind[7]
    }

def evaluate_individual(ind, X_train, X_val, epochs=15, batch_size=64):
    """
    Fitness function: negative validation MSE (higher fitness = better)
    Trains autoencoder for limited epochs to evaluate hyperparameters.
    """
    hparams = individual_to_hparams(ind)
    
    try:
        # Build and train model
        model = build_autoencoder(hparams)
        
        # Early stopping to speed up evaluation
        callbacks = [
            keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5)
        ]
        
        history = model.fit(
            X_train, X_train,
            validation_data=(X_val, X_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=0
        )
        
        # Fitness = negative validation loss (maximization problem)
        fitness = -history.history['val_loss'][-1]
        
        # Clean up to avoid memory leaks
        tf.keras.backend.clear_session()
        del model
        
        return (fitness,)
    
    except Exception as e:
        print(f"⚠️ Training failed for individual {ind}: {str(e)}")
        tf.keras.backend.clear_session()
        return (-1e6,)  # Very bad fitness

def setup_deap_toolbox(X_train, X_val):
    """Configure DEAP toolbox for hyperparameter optimization."""
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    
    toolbox = base.Toolbox()
    toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate_individual, X_train=X_train, X_val=X_val)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutUniformInt, 
                     low=[16,16,3,3,32,32,16,0], 
                     up=[64,64,7,7,128,128,64,1], 
                     indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    return toolbox

def run_genetic_optimization(X_train, X_val, pop_size=30, n_gen=20):
    """Execute GA to find optimal hyperparameters."""
    print(f"\n[GA] Starting hyperparameter optimization ({pop_size} individuals, {n_gen} generations)...")
    
    toolbox = setup_deap_toolbox(X_train, X_val)
    population = toolbox.population(n=pop_size)
    
    # Evaluate initial population
    fitnesses = map(toolbox.evaluate, population)
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit
    
    # Evolution loop
    for gen in range(n_gen):
        print(f"  Generation {gen+1}/{n_gen} | Best fitness: {max(ind.fitness.values[0] for ind in population):.6f}")
        
        # Select parents
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        
        # Crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.7:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        # Mutation
        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        
        # Evaluate invalid individuals
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Replace population
        population[:] = offspring
    
    # Return best individual
    best_ind = tools.selBest(population, 1)[0]
    print(f"\n[GA] Optimization complete. Best fitness: {best_ind.fitness.values[0]:.6f}")
    print(f"Best hyperparameters: {best_ind}")
    
    return individual_to_hparams(best_ind)

# ============================================================================
# 5. ANOMALY DETECTION PIPELINE
# ============================================================================
def train_final_autoencoder(hparams, X_train, X_val, epochs=100, batch_size=64):
    """Train final autoencoder with best hyperparameters."""
    print("\n[Autoencoder] Training final model with optimized hyperparameters...")
    model = build_autoencoder(hparams)
    
    callbacks = [
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1),
        keras.callbacks.ModelCheckpoint('best_autoencoder.h5', monitor='val_loss', save_best_only=True, verbose=0)
    ]
    
    history = model.fit(
        X_train, X_train,
        validation_data=(X_val, X_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=2
    )
    
    # Load best weights
    model.load_weights('best_autoencoder.h5')
    return model, history

def compute_reconstruction_errors(model, X):
    """Compute MSE reconstruction error for each window."""
    reconstructions = model.predict(X, verbose=0)
    errors = np.mean(np.square(X - reconstructions), axis=(1, 2))  # Mean over timesteps and features
    return errors

def determine_anomaly_threshold(errors, percentile=95):
    """Determine threshold using percentile of reconstruction errors."""
    threshold = np.percentile(errors, percentile)
    print(f"[Threshold] Using {percentile}th percentile: {threshold:.6f}")
    return threshold

def detect_anomalies(model, X, threshold):
    """Detect anomalies based on reconstruction error threshold."""
    errors = compute_reconstruction_errors(model, X)
    anomalies = (errors > threshold).astype(int)
    return anomalies, errors

# ============================================================================
# 6. CLASSIFIER (Bidirectional LSTM Bagging Ensemble)
# ============================================================================
def build_classifier(input_shape, lstm_units=64, dense_units=32):
    """Build single Bidirectional LSTM classifier."""
    inputs = layers.Input(shape=input_shape)
    x = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True))(inputs)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)
    x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_bagging_ensemble(X_train, y_train, n_estimators=5, epochs=50):
    """Train bagging ensemble of classifiers on bootstrap samples."""
    print(f"\n[Classifier] Training bagging ensemble ({n_estimators} models)...")
    models = []
    n_samples = len(X_train)
    
    for i in range(n_estimators):
        # Bootstrap sampling
        idx = np.random.choice(n_samples, size=n_samples, replace=True)
        X_boot, y_boot = X_train[idx], y_train[idx]
        
        model = build_classifier((WINDOW_SIZE, N_FEATURES))
        model.fit(
            X_boot, y_boot,
            epochs=epochs,
            batch_size=64,
            verbose=0,
            callbacks=[keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
        )
        models.append(model)
        print(f"  Trained model {i+1}/{n_estimators}")
    
    return models

def ensemble_predict(models, X):
    """Aggregate predictions from ensemble members."""
    preds = np.zeros((len(X),))
    for model in models:
        preds += model.predict(X, verbose=0).flatten()
    return (preds / len(models) > 0.5).astype(int)

# ============================================================================
# 7. EVALUATION & COMPARISON
# ============================================================================
def evaluate_anomaly_detection(y_true, y_pred_autoencoder, y_pred_classifier):
    """Compare autoencoder vs classifier performance."""
    print("\n" + "="*70)
    print("ANOMALY DETECTION EVALUATION")
    print("="*70)
    
    # Autoencoder metrics
    p_ae, r_ae, f1_ae, _ = precision_recall_fscore_support(y_true, y_pred_autoencoder, average='binary')
    auc_ae = roc_auc_score(y_true, y_pred_autoencoder)
    cm_ae = confusion_matrix(y_true, y_pred_autoencoder)
    
    # Classifier metrics
    p_clf, r_clf, f1_clf, _ = precision_recall_fscore_support(y_true, y_pred_classifier, average='binary')
    auc_clf = roc_auc_score(y_true, y_pred_classifier)
    cm_clf = confusion_matrix(y_true, y_pred_classifier)
    
    print("\nAutoencoder (Unsupervised Anomaly Detection):")
    print(f"  Precision: {p_ae:.4f} | Recall: {r_ae:.4f} | F1: {f1_ae:.4f} | AUC: {auc_ae:.4f}")
    print("  Confusion Matrix:")
    print(cm_ae)
    
    print("\nBidirectional LSTM Bagging Ensemble (Supervised Classifier):")
    print(f"  Precision: {p_clf:.4f} | Recall: {r_clf:.4f} | F1: {f1_clf:.4f} | AUC: {auc_clf:.4f}")
    print("  Confusion Matrix:")
    print(cm_clf)
    
    print("\nKey Insight:")
    print("  Autoencoder detects deviations from normal patterns without labels.")
    print("  Classifier leverages labeled anomalies but requires supervision.")
    print("="*70)

# ============================================================================
# 8. MAIN PIPELINE
# ============================================================================
def main_pipeline(df, target_col='class'):
    """
    End-to-end pipeline:
    1. Preprocess data (ignores 'class' during autoencoder training)
    2. Optimize hyperparameters via GA
    3. Train final autoencoder on unlabeled data
    4. Determine anomaly threshold
    5. Train classifier ensemble on labeled portion (for comparison only)
    6. Evaluate both approaches on test set
    """
    print("="*70)
    print("SPACECRAFT TELEMETRY ANOMALY DETECTION PIPELINE")
    print("="*70)
    
    # Step 1: Preprocess
    X_train, X_val, X_test, y_test, scaler = preprocess_data(df, target_col=target_col)
    print(f"\n[Data] Shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    if y_test is not None:
        print(f"[Data] Test set anomalies: {np.sum(y_test)} / {len(y_test)} ({np.mean(y_test)*100:.2f}%)")
    
    # Step 2: Hyperparameter optimization (using only unlabeled data)
    best_hparams = run_genetic_optimization(X_train, X_val)
    
    # Step 3: Train final autoencoder
    autoencoder, history = train_final_autoencoder(best_hparams, X_train, X_val)
    
    # Step 4: Determine anomaly threshold on validation set
    val_errors = compute_reconstruction_errors(autoencoder, X_val)
    threshold = determine_anomaly_threshold(val_errors, percentile=95)
    
    # Step 5: Detect anomalies on test set
    y_pred_ae, test_errors = detect_anomalies(autoencoder, X_test, threshold)
    
    # Step 6: Train classifier ensemble (ONLY for comparison - uses labels)
    if y_test is not None:
        # Create training labels for classifier (using last timestep of each window)
        y_train_windows = create_windows(df[target_col].values.reshape(-1, 1))
        y_train = y_train_windows[:len(X_train), -1, 0].astype(int)
        
        # Train ensemble
        classifier_ensemble = train_bagging_ensemble(X_train, y_train, n_estimators=5)
        y_pred_clf = ensemble_predict(classifier_ensemble, X_test)
        
        # Step 7: Evaluate both approaches
        evaluate_anomaly_detection(y_test, y_pred_ae, y_pred_clf)
    else:
        print("\n⚠️ No labels available for test set - skipping classifier comparison.")
        print(f"Detected {np.sum(y_pred_ae)} anomalies in test set using autoencoder.")
    
    # Cleanup
    tf.keras.backend.clear_session()
    print("\n[Pipeline] Completed successfully.")

# ============================================================================
# USAGE EXAMPLE (user loads dataset themselves)
# ============================================================================
if __name__ == "__main__":
    # User should load their dataset here:
    # import pandas as pd
    # df = pd.read_csv('your_telemetry_dataset.csv')
    # 
    # Then run:
    main_pipeline(dataset, target_col='Сlass')
    #
    # Requirements:
    #   pip install numpy tensorflow scikit-learn deap
    
  